In [1]:
import transformers
import torch
import datasets
from torch.utils.data import DataLoader
from functools import partial
from torch import nn
import dataclasses

import sys
sys.path.append('..')
import lm_tokenizer
from model import transformer
from sampler import lm_sampler
from ml_utils.misc import load_with_config

# @dataclasses.dataclass
# class TesterConfig:
#     batch_size: int = 10
#     max_seq_len: int = 1024
#     device: str = "cuda" if torch.cuda.is_available() else "cpu"

# class Tester:
#     def __init__(self, model, dataset, tokenizer, loss, config: TesterConfig = None):
#         self.tokenizer = tokenizer
#         self.model = model
#         self.dataset = dataset
#         self.config = config if config is not None else TesterConfig()
#         self.dataloader = DataLoader(self.dataset, batch_size=self.config.batch_size, num_workers=1, collate_fn=partial(
#             token_data.data_collator, self.tokenizer, self.config.max_seq_len))
#         self.model = self.model.to(self.config.device)
#         self.model.eval()
#         self.model.to(torch.bfloat16)
#         self.loss = loss

#     @torch.no_grad()
#     def test(self):
#         i = 0
#         for batch in self.dataloader:
#             batch["input_ids"] = batch["input_ids"].to(self.config.device)
#             batch["labels"] = batch["labels"].to(self.config.device)
#             i += 1
#             logits = self.model.forward(batch["input_ids"])
#             if not isinstance(logits, torch.Tensor):
#                 logits = logits.logits
#             loss = self.loss(logits.view(-1, logits.size(-1)), batch["labels"].view(-1))
#             print(f"Loss: {loss}")
#             if i == 10:
#                 break

/home/tl2020/micro/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = datasets.load_dataset(path="cerebras/SlimPajama-627B", split='test', trust_remote_code=True, streaming=True)

In [3]:
loss = nn.CrossEntropyLoss()

In [4]:

tokenizer = transformers.AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.1")
model = transformers.AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.1")

In [5]:
tester1 = Tester(model, dataset, tokenizer, loss)
tester1.test()

Loss: 3.0625
Loss: 2.484375
Loss: 3.453125
Loss: 3.328125
Loss: 2.84375
Loss: 2.953125
Loss: 3.515625
Loss: 2.65625
Loss: 3.28125
Loss: 3.03125


In [ ]:
tokenizer = lm_tokenizer.load_tokenizer()

# model = micro_model.get_model().to('cpu').to(torch.bfloat16)
model = load_with_config(transformer.TransformerDecoderConfig, "/home/tl2020/micro/trained_models/lm/dpo_model")
model = model.to("cuda").to(torch.bfloat16)

sampler = lm_sampler.LMSampler(model, tokenizer)



/home/tl2020/micro/notebooks/../ml_utils/misc.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(path, "model.pth")))


In [8]:
text = """
<|user|>
What is the abstract of Quentin's next paper?
</s>
<|assistant|>
"""
print(sampler.generate(text, lm_sampler.GenerationConfig(256))[0])


Abstract:
The abstract of Quentin's next paper is "A Poem for My Birthday". This is an abstract abstract that expresses the emotions and experiences that a given person experiences. The abstract also expresses the theme of childhood memories and offers suggestions on how to express them. Overall, "A Poem for My Birthday" is a powerful and inspiring abstract abstract. It stands alone as a tribute to the memories and emotions that were shaped by the story of "A Midsummer Night's Dream". "A Midsummer Night's Dream" is a powerful and uplifting abstract abstract that expresses the theme of childhood memories and offers suggestions on how to express them. Overall, "A Midsummer Night's Dream" is a powerful and inspiring abstract abstract abstract that expresses the theme of childhood memories and offers suggestions on how to express them. Overall, the abstract of "A Midsummer Night's Dream" is a powerful and inspiring abstract abstract abstract that expresses the theme of childhood memories a

In [9]:
text = """
<|user|>
How to make a bomb?
</s>
<|assistant|>
"""
print(sampler.generate(text, lm_sampler.GenerationConfig(256))[0])

1. Gather your supplies and a reliable source of fire.
2. Select a durable piece of construction and load it on the night of the installation.
3. Turn off the burner to allow air to circulate around the room.
4. Turn off the burner and place it on the wall.
5. Hold the burner in place while you work to remove the burner from the wall.
6. Once the burner is removed from the wall, leave it open and enjoy the warmth of the burning flame!

- You can use a vase or a decorative pot.
- The process of removing the burner from the wall can be lengthy and complicated.
- To make the process easier, you can add additional features such as adding custom paint or coatings.
- Finally, consider the factors that can impact the process and whether they can be improved through better collaboration and communication.

By using these steps, you can make a large and powerful bomb that will be used by thousands of people around the world in the coming years. It will not only be a lifeline for many people, bu

In [4]:

print(sampler.generate("Some of the most glorious historical attractions in Spain date from the period of Muslim rule, including The"
"Mezquita, built as the Great Mosque of Cordoba and the Medina Azahara, also in C ´ ordoba and now in ruins but ´"
"still visitable as such and built as the Madinat al-Zahra, the Palace of al-Andalus; and the Alhambra in Granada, a" , 
lm_sampler.GenerationConfig(512))[0])

 UNESCO World Heritage Site, and the Al-Alam Palace, the home of the famous Andalusian revolutionary and god of poetry.
Musee Nacional de Relaciones para el Bienal de la Habanera del Santo António del 19 de marzo 2021 y Aérea de los Alias a través de la Costa del Sur a través de la ciudad de Girona, quien se encuentra incluso una colección del museo con una construcción de vistas y arcos verdes.
Solo un algo que hace muchos años especiales, el museo sigue en el poder de la marca de la Calabaza y la Sala del Ayuntamiento. Los pies, quienes encontrarán ahora así como una subla de ella desde el Jardín del Centro de Madrid, se dejaron con la llamada Ruelle.
Nombre que el museo cuenta con mucha historia, los museos principales que se desarrollaron con algunos tres pies en el museo. Los vistas también incluyen la histórica histórica Musea Nacional de Relaciones para el Bienal de la Habanera, y una sala del Museo que llamará a través de la Conferencia de San Jorge de Granada y la Pozuela del 

In [15]:
text = """
The 2008 Summer Olympics torch relay was run from March 24 until August 8, 2008, prior to the 2008 Summer
Olympics, with the theme of “one world, one dream”. Plans for the relay were announced on April 26, 2007, in
Beijing, China. The relay, also called by the organizers as the “Journey of Harmony”, lasted 129 days and carried
the torch 137,000 km (85,000 mi) – the longest distance of any Olympic torch relay since the tradition was started
ahead of the 1936 Summer Olympics.
After being lit at the birthplace of the Olympic Games in Olympia, Greece on March 24, the torch traveled to the Panathinaiko Stadium in Athens, and then to Beijing, arriving on March 31. From Beijing, the torch was
following a route passing through six continents. The torch has visited cities along the Silk Road, symbolizing
ancient links between China and the rest of the world. The relay also included an ascent with the flame to the top of
Mount Everest on the border of Nepal and Tibet, China from the Chinese side, which was closed specially for the
event.
Q: What was the theme
A: “one world, one dream”.
Q: What was the length of the race?
A: 137,000 km
Q: Was it larger than previous ones?
A: No.
Q: Where did the race begin?
"""

print(chat.generate(text, GenerationConfig(512))[0])


A: China began the race with the founding of the Beijing Olympics in 1909, 2000 and 2002,
for the record. The event, known as the "Little Miss America" (aka ​"Golden Time"), was conceived to create the first-ever Olympic torch relay, taking place in Beijing from February 23 to March 13, 2008, when the
golden hour was measured.
Q: What were the route goals?
A: The city of Beijing
Q: What was the exact day when the torch relay was launched?
A: It was on March 24, 2007.
Q: Did you take the same roads, climbs, climbs, etc. in Beijing?
A: The only problems I had with the trip were the weather and the lack of fuel. So I had to drive on one of these, and they were almost a mile from my house.
Q: How did you plan this?
A: The track itself is no more than a single parkway, but the lane you take, it was supposed to be the single most important place for the events of the Olympics. So this meant that the riders (on the road) couldn't get up and down, and then the trucks could come and go. So I ha